# Importing packages

In [1]:
# Import data reader to directly convert Yahoo Finance data into dataframe
import yfinance as yf
import yahoo_fin
from yahoo_fin import options

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
import seaborn as sns
from datetime import datetime
from functools import reduce

# Import data reader to directly convert Yahoo Finance data into dataframe
from pandas_datareader import data as pdr
yf.pdr_override()

from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.stats
from scipy.stats import norm
from math import log, sqrt, pi, exp

sns.set_style('darkgrid')

style.use('fivethirtyeight')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Collecting data

In [ ]:
def stock_prices_returns(ticker, start_date, end_date):
    global stock
    stock = pdr.get_data_yahoo(ticker, start = start_date, end = end_date)[['Adj Close']]
    stock['return'] = stock.pct_change()
    stock.dropna(inplace = True)
    return stock

tickerSymbol = 'AMZN'

stock_prices_returns(tickerSymbol, '2021-06-01', '2022-04-15')

In [ ]:
#plot
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(stock['Adj Close'], color = 'turquoise')
ax.set(title = tickerSymbol, ylabel = 'Price per share')

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(8,5))
ax.hist(stock['return'], color='salmon', bins=40)
ax.set(title='Distribution of Returns')

The distribution of returns is almost normal except for a slight skew and fatter tail, hence Black Scholes assumption of normally distributed returns 

## Options Data

In [ ]:
#Get expiration dates
expiration_dates = options.get_expiration_dates(tickerSymbol)
expiration_dates

In [7]:
# Get options data for each expiration date
expiration_dates= options.get_expiration_dates(tickerSymbol)

options_chain = {}
for date in expiration_dates:
    options_chain[date] = options.get_options_chain(tickerSymbol)

In [ ]:
# Options info for June 20,2022 expiration call options
calls = options_chain['June 20, 2022']['calls']
calls

In [ ]:
# Options info for June 20,2022 expiration put options
puts = options_chain['June 20, 2022']['puts']
puts

## Black Scholes

### Parameters

In [ ]:
# Spot price S (current price)

S = stock['Adj Close'].iloc[-1]
S

In [11]:
# Strike price X 
X = 142

In [ ]:
# time to Maturity(T)
expiry = '17-07-2022'
today = datetime.now()
T = (datetime.strptime(expiry, "%d-%m-%Y") - today).days/365
T

In [ ]:
# Risk-Free rate (r), we will use the US Treasury Yield '^TNX'
r = (pdr.get_data_yahoo('^TNX')['Adj Close'].iloc[-1]) / 100
r

In [ ]:
# Volatility (sigma)

def sigma(df):
  daily_volatility = df['return'].std()
  yearly_trade_days = 252
  sigma = np.sqrt(yearly_trade_days) * daily_volatility
  return sigma

sigma(stock)

In [ ]:
## Combining all parameters in one cell

# S is the current spot price of underlying stock
S = stock['Adj Close'].iloc[-1]

# X is the option strike price
# T is the time until maturity (in fractions of a year)

today = datetime.now()
T = (datetime.strptime(expiry, "%d-%m-%Y") - datetime.now()).days / 365

# r is the risk free rate, which we'll use as the 10-year U.S. Treasury Yield '^TNX'
today = datetime.now()
r = (pdr.get_data_yahoo('^TNX')['Adj Close'].iloc[-1]) / 100

# sigma is the yearly returns volatility of the underlying stock
daily_volatility = stock['return'].std()
yearly_trade_days = 252
sigma = np.sqrt(yearly_trade_days) * daily_volatility

## Defining Black Scholes function

In [16]:
# d1 is the probability of receiving the stock at expiration 
def d11(S, X, T, r, sigma):
    return (np.log(S/X) + (r + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))

# d2 is the risk-adjusted probability that the option will be exercised
def d21(d1, T, sigma):
    return d1 - sigma * np.sqrt(T)

# Black-Scholes European option pricing formula    
def black_scholes(S, X, T, r, sigma, option_type):
    global d_one, d_two
    d_one = d11(S, X, T, r, sigma)
    d_two = d21(d_one, T, sigma)
    if option_type == 'call':
        return S * norm.cdf(d_one) - np.exp(-r * T) * X * norm.cdf(d_two)
    elif option_type == 'put':
        return -(S * norm.cdf(-d_one) - np.exp(-r * T) * X * norm.cdf(-d_two))
    else:
        # Raise an error if the option_type is neither a call nor a put
        raise ValueError("Option type is neither 'call' or 'put'.")

## Option Premiums

In [ ]:
# Last 100 spot prices
stock_spot = stock['Adj Close'][-100:]

# Initialize the European put option values array
call_option_values = np.zeros(stock_spot.size)
put_option_values = np.zeros(stock_spot.size)

# Iterate through spot prices and compute the option values
for i,S in enumerate(stock_spot.values):
    call_option_values[i] = black_scholes(S = S, X = X, T = T, r = r, sigma = sigma, option_type = 'call')
    put_option_values[i] = black_scholes(S = S, X = X, T = T, r = r, sigma = sigma, option_type = 'put')

options_values=pd.DataFrame({'spot_price':stock_spot, 'call_option_value':call_option_values, 'put_option_value':put_option_values})

plt.figure(figsize=(12,6))
sns.lineplot(data=options_values['spot_price'], color='turquoise')
sns.lineplot(data=options_values['call_option_value'], color='gold')
sns.lineplot(data=options_values['put_option_value'], color='lightcoral')
plt.ylabel('Price', fontsize=14)
plt.title('Stock Option Values at $142 Strike Expiring Nov 17, 2023', fontsize=18)
plt.legend(labels=['Spot','Call','Put'])

In [ ]:
# Call option premium
price_call = black_scholes(S = S, X = X, T = T, r = r, sigma = sigma, option_type = 'call')
price_call

In [ ]:
# Put option premium
price_put = black_scholes(S = S, X = X, T = T, r = r, sigma = sigma, option_type = 'put')
price_put

## Implied Volatility

In [20]:
def call_implied_volatility(Price, S, X, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = black_scholes(S=S, X=X, T=T, r=r, sigma=sigma, option_type='call')
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, X, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = black_scholes(S=S, X=X, T=T, r=r, sigma=sigma, option_type='put')
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

In [ ]:
print("Put Implied Volatility: " + str(round(100 * put_implied_volatility(puts[puts['Strike']==X]['Last Price'].values[0], S, X, T, r),2))+ " %")

## Volatility smile

In [ ]:
strikes = puts['Strike']

# Initialize implied volatilty (IV) array
put_IV = np.zeros(strikes.size)

# Iterate through strike prices and compute the implied volatility
for i,X in enumerate(strikes.values):
  put_IV[i] = put_implied_volatility(puts[puts['Strike']==X]['Last Price'].values[0], S, X, T, r)
  
IV_values=pd.DataFrame({'strike_price':strikes, 'put_implied_volatility':put_IV})

plt.figure(figsize=(8,6))
sns.scatterplot(data=IV_values, x='strike_price', y='put_implied_volatility', color='turquoise', s=60)
plt.xlabel('Strike Price ($)', fontsize=14)
plt.ylabel('Implied Volatility', fontsize=14)
plt.title('Implied Volatility for Put Option Expiring July 17, 2022', fontsize=18)

## Greeks

In [ ]:
# Set parameters again as we iterated through them before
S = stock['Adj Close'].iloc[-1]
X = 142
T = (datetime.strptime(expiry, "%d-%m-%Y") - datetime.now()).days / 365
r = (pdr.get_data_yahoo('^TNX')['Adj Close'].iloc[-1]) / 100
sigma = np.sqrt(yearly_trade_days) * daily_volatility

def put_greeks(S, X, T, r, sigma, d1):
  # Returns dataframe of greeks 

  global greeks_df

  delta = -norm.cdf(-d11(S,X,T,r,sigma))
  gamma = norm.pdf(d11(S,X,T,r,sigma))/(S*sigma*sqrt(T))
  vega = 0.01*(S*norm.pdf(d11(S,X,T,r,sigma))*sqrt(T))
  theta = 0.01*(-(S*norm.pdf(d11(S,X,T,r,sigma))*sigma)/(2*sqrt(T)) + r*X*exp(-r*T)*norm.cdf(-d21(d1,T,sigma)))
  rho = 0.01*(-X*T*exp(-r*T)*norm.cdf(-d21(d1,T,sigma)))

  greeks_dict = {'Delta: change in option price with $1 change in underlying asset price (velocity)':delta, 
                 'Gamma: change in delta with $1 change in underlying asset price (acceleration)':gamma, 
                 'Vega: change in option price with 1% change in implied volatility':vega,
                 'Theta: change in option price with 1 day change toward expiration (time decay)':theta,
                 'Rho: change in option price with 1% change in interest rate':rho}
  greeks_df = pd.DataFrame.from_dict(greeks_dict, orient='index', columns=['Greeks'])
  return greeks_df

put_greeks(S=S, X=X, T=T, r=r,sigma=sigma,d1=d_one)

In [24]:
# Convert string dates to datetime and then to annualized form
def convert_timestamp(x, from_pattern, to_pattern):
  expiration_dates = datetime.strptime(x, from_pattern)
  return datetime.strftime(expiration_dates, to_pattern)

expiration_dates = [convert_timestamp(x, '%B %d, %Y', '%m-%d-%Y') for x in expiration_dates]
expiration_dates = [((datetime.strptime(T, "%m-%d-%Y") - datetime.now()).days / 365) for T in expiration_dates]
expiration_dates = pd.Series(expiration_dates)

In [ ]:
# Initialize delta array
put_delta = np.zeros(expiration_dates.size)

# Iterate through strike prices and compute the implied volatility
for i,T in enumerate(expiration_dates.values):
  put_delta[i] = -norm.cdf(-d11(S,X,T,r,sigma))
  
delta_values=pd.DataFrame({'expiration_date':expiration_dates, 'delta':put_delta})

plt.figure(figsize=(8,6))
sns.scatterplot(data=delta_values, x='expiration_date', y='delta', color='crimson', s=150)
plt.xlabel('Time to Expiry', fontsize=14)
plt.ylabel('Delta', fontsize=14)
plt.title('Delta for Put Option by Time to Expiry', fontsize=18)

## Delta Hedging

In [ ]:
S = stock['Adj Close'].iloc[-1]
X = 142
T = (datetime.strptime(expiry, "%d-%m-%Y") - datetime.now()).days / 365
r = (pdr.get_data_yahoo('^TNX')['Adj Close'].iloc[-1]) / 100
sigma = np.sqrt(yearly_trade_days) * daily_volatility

def delta_neutral(shares):
  # returns statement of number of options needed for delta hedge based on number of shares of underlying asset

  asset_delta = 1
  asset_total_delta = shares * asset_delta
  option_delta = greeks_df.iloc[0,0]
  option_total_delta = -asset_total_delta
  shares_per_option = 100

  global n
  n = option_total_delta / option_delta / shares_per_option 
  
  return print('We need to buy {} put options for a delta-neutral position.'.format(round(n)))


delta_neutral(100)

Our delta neutral portfolio would be 100 shares of AMZN and 2 put option

In [ ]:
# Market price of put option at our strike price
option_price = puts[puts['Strike']==X]['Last Price'].values[0]
print('Price of put options contract is ${}'.format(option_price))
print('Option contract value (cash outlay) to delta hedge AMZN is ${}'.format(round(n * option_price * 100),2))

Waht if AMZN price declined by $1?

In [ ]:
value = black_scholes(S = S, X = X, T = T, r = r, sigma = sigma, option_type = 'put')
option_delta = greeks_df.iloc[0,0]

# Find the option value change when the price decrease by 1
value_change = black_scholes(S = S-1, X = X, T = T, r = r, sigma = sigma, option_type = 'put') - value

# Total change of both stock and option
-1 + (value_change / abs(option_delta))